Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
import pyspark.sql.functions as F

In [3]:
# создание Spark сессии
spark = SparkSession.builder \
    .appName("Sales Analysis") \
    .getOrCreate()

24/06/02 23:20:29 WARN Utils: Your hostname, MateBook-D16 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
24/06/02 23:20:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/02 23:20:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# данные
data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]


In [5]:
# создание DataFrame
df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])


In [6]:
df

DataFrame[date: string, category: string, quantity: bigint, revenue: bigint]

In [7]:
# оконная функция для расчета средней выручки по категориям
window = Window.partitionBy("category")

In [8]:
# добавление столбца с средней выручкой по категории
df = df.withColumn("avg_revenue", F.avg("revenue").over(window))

In [10]:
# применение операции pivot
pivot_df = df.groupBy("category").pivot("date").agg(F.first("avg_revenue")).show()
